# 1. Import

In [1]:
!pip install langchain langchain-openai tiktoken openai faiss-cpu pypdf chromadb langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.4 MB/s eta 0:00:

In [23]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chains import LLMChain

In [20]:
# llm 모델 설정
llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.7,
    streaming=True # 스트리밍 출력
)

In [31]:
prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 광고 카피라이터입니다. 주어진 카드 정보와 혜택을 바탕으로 기발하고 명확한 카피라이팅 문구를 제작하는 것이 역할입니다.
당신의 문구는 사용자에게 카드의 가치를 한눈에 전달할 수 있어야 하며, 읽는 사람이 혜택을 즉각적으로 이해하고 매력을 느끼게 해야 합니다.
이모티콘을 더해서 산뜻한 느낌을 더해주세요.
<|eot_id|><|start_header_id|>user<|end_header_id|>

카드 정보:
{llm_input}

요청사항:
입력된 카드 정보 각각에 대해 2~3개의 기발한 광고 문구를 작성해주세요.

예시 문구:
1. "영화와 캐시백의 만남 – 삼성카드 MILEAGE PLATINUM이 선사합니다!"
2. "와이즈한 선택, 국민 와이즈카드로 누리는 뷰티와 문화의 향연!"

위와 같은 스타일로 작성해주세요.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(input_variables=["llm_input"],template=prompt_template)

# LLM Chain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt)

llm_input = """
- 카드 이름: KB국민 와이즈카드
- 혜택: 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아
"""

response = llm_chain.run({"llm_input":llm_input})

# 결과 출력
print(response)

1. "뷰티에 대한 사랑, KB국민 와이즈카드로 더 빛나게! 💄✨"
2. "납부도 스마트하게! 국민 와이즈카드로 간편하게 해결하세요! 💳👍"

3. "카페와 베이커리의 달콤한 혜택, 국민 와이즈카드와 함께! ☕🍰"
4. "커피 한 잔의 여유, 국민 와이즈카드가 더해주는 특별한 순간! 🎉☕"

5. "쇼핑의 즐거움, 간편결제로 더욱 스마트하게! 🛍️💳"
6. "국민 와이즈카드와 함께라면 쇼핑이 더 쉽고 즐거워요! 🎈🛒"

7. "의료비 걱정 없이 건강하게! 국민 와이즈카드가 함께합니다! 🏥❤️"
8. "내 건강, 내 선택! 국민 와이즈카드로 더 똑똑한 의료비 절약! 🩺✨"

9. "교통과 주유의 스마트한 동반자, 국민 와이즈카드와 함께! 🚗⛽"
10. "편리한 이동, 알뜰한 주유! 국민 와이즈카드가 도와드립니다! 🌍💨"

11. "문화생활의 풍요로움, 국민 와이즈카드로 누려보세요! 🎭🎨"
12. "예술과 여유를 동시에! 국민 와이즈카드가 여러분을 초대합니다! 🌟🎟️"

13. "교육과 육아의 든든한 지원군, 국민 와이즈카드와 함께! 📚👶"
14. "아이의 미래를 위한 현명한 선택, 국민 와이즈카드가 함께합니다! 🌈👩‍👧‍👦"
